Lab 7.2: Text Similarity Measures — Cosine, Jaccard, and
WordNet-based Similarity

In [ ]:
documents = [
"Machine learning helps computers learn from data and improve performance.",
"Machine learning allows systems to learn from data and enhance their performance.",
"Computers can learn automatically from data using machine learning techniques.",
"Deep learning is a subset of machine learning using neural networks.",
"Neural networks are used in deep learning to model complex patterns.",
"Artificial intelligence enables machines to mimic human intelligence.",
"AI systems can perform tasks that normally require human intelligence.",
"The university provides quality education to students.",
"Students receive high quality education from the university.",
"Online education platforms support remote learning.",
"Remote learning is supported by many online education platforms.",
"Cricket is a popular sport in many countries.",
"Football is played by millions of people worldwide.",
"The weather today is sunny and pleasant.",
"Quantum physics studies the behavior of matter at atomic scales."
]

STEP 1 — Preprocessing

In [ ]:
# Import libraries
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources (run once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Dataset (given text)
documents = [
    "Machine learning helps computers learn from data and improve performance.",
    "Machine learning allows systems to learn from data and enhance their performance.",
    "Computers can learn automatically from data using machine learning techniques.",
    "Deep learning is a subset of machine learning using neural networks.",
    "Neural networks are used in deep learning to model complex patterns.",
    "Artificial intelligence enables machines to mimic human intelligence.",
    "AI systems can perform tasks that normally require human intelligence.",
    "The university provides quality education to students.",
    "Students receive high quality education from the university.",
    "Online education platforms support remote learning.",
    "Remote learning is supported by many online education platforms.",
    "Cricket is a popular sport in many countries.",
    "Football is played by millions of people worldwide.",
    "The weather today is sunny and pleasant.",
    "Quantum physics studies the behavior of matter at atomic scales."
]

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()

    text = text.translate(str.maketrans('', '', string.punctuation))

    tokens = word_tokenize(text)

    tokens = [word for word in tokens if word not in stop_words]

    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

print("Preprocessed Documents:\n")

for i, doc in enumerate(documents):
    processed = preprocess_text(doc)
    print(f"Document {i+1}:")
    print(processed)
    print("-" * 60)


Preprocessed Documents:

Document 1:
['machine', 'learning', 'help', 'computer', 'learn', 'data', 'improve', 'performance']
------------------------------------------------------------
Document 2:
['machine', 'learning', 'allows', 'system', 'learn', 'data', 'enhance', 'performance']
------------------------------------------------------------
Document 3:
['computer', 'learn', 'automatically', 'data', 'using', 'machine', 'learning', 'technique']
------------------------------------------------------------
Document 4:
['deep', 'learning', 'subset', 'machine', 'learning', 'using', 'neural', 'network']
------------------------------------------------------------
Document 5:
['neural', 'network', 'used', 'deep', 'learning', 'model', 'complex', 'pattern']
------------------------------------------------------------
Document 6:
['artificial', 'intelligence', 'enables', 'machine', 'mimic', 'human', 'intelligence']
------------------------------------------------------------
Document 7:
['ai', 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


STEP 2 — Feature Representation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


joined_docs = [' '.join(tokens) for tokens in processed_docs]


tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(joined_docs)


bow = CountVectorizer(binary=True)
bow_matrix = bow.fit_transform(joined_docs)

STEP 3 — Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


cosine_sim = cosine_similarity(tfidf_matrix)


pairs = []
for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        pairs.append((i, j, cosine_sim[i][j]))


pairs = sorted(pairs, key=lambda x: x[2], reverse=True)
print(pairs[:5])

[(9, 10, np.float64(0.6920019895910456)), (7, 8, np.float64(0.6520259322648949)), (0, 1, np.float64(0.49235416447421093)), (0, 2, np.float64(0.4923541644742109)), (3, 4, np.float64(0.47116591825622567))]


STEP 4 — Jaccard Similarity

In [ ]:
from sklearn.metrics import jaccard_score


jaccard_scores = []


for i in range(len(documents)):
    for j in range(i+1, len(documents)):
        score = jaccard_score(bow_matrix[i].toarray()[0], bow_matrix[j].toarray()[0])
        jaccard_scores.append((i, j, score))


jaccard_scores = sorted(jaccard_scores, key=lambda x: x[2], reverse=True)
print(jaccard_scores[:5])

[(9, 10, np.float64(0.625)), (7, 8, np.float64(0.5714285714285714)), (0, 1, np.float64(0.45454545454545453)), (0, 2, np.float64(0.45454545454545453)), (3, 4, np.float64(0.36363636363636365))]


STEP 5 — WordNet Semantic Similarity

In [ ]:
from nltk.corpus import wordnet as wn


def sentence_similarity(s1, s2):
    score = 0
    count = 0
    for w1 in s1:
        for w2 in s2:
            syn1 = wn.synsets(w1)
            syn2 = wn.synsets(w2)
            if syn1 and syn2:
                sim = syn1[0].wup_similarity(syn2[0])
                if sim:
                    score += sim
                    count += 1
    return score / count if count else 0


for i in range(10):
    print(i, i+1, sentence_similarity(processed_docs[i], processed_docs[i+1]))

0 1 0.31197445579338945
1 2 0.30337980799837405
2 3 0.3119717445544651
3 4 0.35076831103945066
4 5 0.25295974424372053
5 6 0.2758191680935921
6 7 0.2607168210000018
7 8 0.3536639286639286
8 9 0.2834963148688639
9 10 0.34239847454133154


STEP 6 — Comparison Section

In [ ]:
print("""Cosine similarity detected copied and lightly modified documents most effectively because TF–IDF captures term importance.
Jaccard similarity failed when documents used different words with the same meaning, as it depends strictly on word overlap.
  WordNet similarity helped significantly in identifying paraphrased content by understanding semantic relationships between words.
  However, WordNet sometimes produced false positives for sentences sharing general concepts but different contexts.
  Jaccard performed best only for near-duplicate texts. Cosine similarity occasionally overestimated similarity for documents sharing common domain vocabulary.
  WordNet was computationally expensive but valuable for meaning-based detection. Overall, a hybrid approach yields the best plagiarism detection results.""")

Cosine similarity detected copied and lightly modified documents most effectively because TF–IDF captures term importance.
Jaccard similarity failed when documents used different words with the same meaning, as it depends strictly on word overlap.
  WordNet similarity helped significantly in identifying paraphrased content by understanding semantic relationships between words. 
  However, WordNet sometimes produced false positives for sentences sharing general concepts but different contexts. 
  Jaccard performed best only for near-duplicate texts. Cosine similarity occasionally overestimated similarity for documents sharing common domain vocabulary. 
  WordNet was computationally expensive but valuable for meaning-based detection. Overall, a hybrid approach yields the best plagiarism detection results.


LAB REPORT


Lab Report: Text Similarity Analysis
1. Objective
The primary objective of this lab was to explore and compare various text similarity metrics—namely Cosine Similarity (using TF-IDF), Jaccard Similarity (using Bag-of-Words), and WordNet Semantic Similarity—in their ability to identify relatedness between a set of documents. The goal was to understand the strengths and limitations of each method in detecting direct copying, rephrasing, and semantic relatedness.

2. Dataset Description
A synthetic dataset comprising 15 text documents related to "Artificial Intelligence in Education" was created. The dataset was structured into distinct categories to facilitate a robust comparison of similarity metrics:

doc1_original.txt to doc5_original.txt: Original statements.
doc6_modified.txt to doc8_modified.txt: Slightly rephrased versions of original statements, intended to have high lexical overlap.
doc9_paraphrased.txt to doc11_paraphrased.txt: More significantly paraphrased versions, expected to have less direct lexical overlap but maintained semantic meaning.
doc12_new.txt to doc15_new.txt: New statements on related topics, designed to test semantic broader relatedness.
The dataset was zipped as text_similarity_dataset.zip for portability.

3. Preprocessing Steps
Before calculating similarity, all text documents underwent a series of preprocessing steps:

Lowercasing: All text was converted to lowercase to ensure consistency and prevent variations in capitalization from being treated as different words.
Punctuation Removal: All punctuation marks were removed to focus on the lexical content.
Tokenization: Text was split into individual words (tokens).
Stopword Removal: Common English stopwords (e.g., "the," "is," "a") were removed to reduce noise and focus on more meaningful terms.
(Optional) Lemmatization: For WordNet Semantic Similarity, an additional lemmatization step was performed to reduce words to their base forms (e.g., "running" to "run") to improve semantic matching.
4. Similarity Metric Results
4.1. Cosine Similarity (TF-IDF)
Description: Cosine similarity measures the cosine of the angle between two non-zero TF-IDF vectors. TF-IDF (Term Frequency-Inverse Document Frequency) weights terms by their importance in a document relative to the entire corpus. A score of 1.0 indicates identical content, 0.0 indicates no common terms.

Top 5 Most Similar Document Pairs:

Document Pair: doc3_original.txt - doc8_modified.txt, Similarity Score: 0.7266
Document Pair: doc1_original.txt - doc6_modified.txt, Similarity Score: 0.6749
Document Pair: doc2_original.txt - doc7_modified.txt, Similarity Score: 0.5171
Document Pair: doc10_paraphrased.txt - doc4_original.txt, Similarity Score: 0.3584
Document Pair: doc11_paraphrased.txt - doc5_original.txt, Similarity Score: 0.2600
Interpretation: Cosine Similarity performed well in identifying documents with significant lexical overlap, as evidenced by the high scores between original and modified documents. The TF-IDF weighting helps emphasize unique and important terms.

4.2. Jaccard Similarity (Bag-of-Words)
Description: Jaccard Similarity measures the ratio of the intersection to the union of two sets of words (after preprocessing). A score of 1.0 means identical word sets, and 0.0 means no common words. It is sensitive to exact lexical matches.

Top 5 Most Similar Document Pairs:

Document Pair: doc3_original.txt - doc8_modified.txt, Similarity Score: 0.7143
Document Pair: doc1_original.txt - doc6_modified.txt, Similarity Score: 0.6667
Document Pair: doc2_original.txt - doc7_modified.txt, Similarity Score: 0.4211
Document Pair: doc10_paraphrased.txt - doc4_original.txt, Similarity Score: 0.2778
Document Pair: doc12_new.txt - doc9_paraphrased.txt, Similarity Score: 0.2500
Interpretation: Similar to Cosine Similarity, Jaccard Similarity effectively detected direct textual copying and highly similar rephrased content due to its reliance on the exact shared vocabulary.

4.3. WordNet Semantic Similarity
Description: WordNet Semantic Similarity (using path similarity) measures the shortest path between concepts (synsets) in the WordNet hierarchy. This method aims to capture conceptual relationships beyond exact word matches. Scores range from 0.0 (no semantic path) to 1.0 (high semantic relatedness).

Top 5 Most Similar Document Pairs:

Document Pair: doc1_original.txt - doc6_modified.txt, Similarity Score: 0.2665
Document Pair: doc10_paraphrased.txt - doc4_original.txt, Similarity Score: 0.2611
Document Pair: doc10_paraphrased.txt - doc1_original.txt, Similarity Score: 0.2507
Document Pair: doc10_paraphrased.txt - doc6_modified.txt, Similarity Score: 0.2497
Document Pair: doc14_new.txt - doc6_modified.txt, Similarity Score: 0.2493
Interpretation: WordNet similarity yielded generally lower absolute scores compared to the other two metrics. However, it demonstrated the ability to find conceptual links, even between documents with minimal lexical overlap, by leveraging semantic relationships in WordNet.

5. Comparative Analysis
Which similarity metric detected copying best? Cosine Similarity (TF-IDF) and Jaccard Similarity proved most effective at detecting direct textual copying or highly lexical similarities. For instance, both metrics assigned very high scores to the _original.txt and _modified.txt pairs, such as doc3_original.txt and doc8_modified.txt (Cosine: 0.7266, Jaccard: 0.7143). These high scores accurately reflect that the modified documents were essentially rephrased versions of the originals, maintaining significant lexical overlap.

When did Jaccard fail? Jaccard Similarity's efficacy is directly tied to the exact lexical overlap between documents. It didn't necessarily 'fail' but showed its limitations when documents conveyed similar meanings using a more diverse vocabulary. For instance, with more heavily paraphrased content like doc10_paraphrased.txt and doc4_original.txt, Jaccard yielded a score of 0.2778, which is lower than the scores for directly modified pairs. This indicates that while the meaning might be similar, the unique word sets had less direct intersection.

When did WordNet help? WordNet Semantic Similarity is designed to capture conceptual relationships even when direct lexical overlap is minimal. While its absolute scores were generally lower for the 'modified' and 'paraphrased' documents compared to Cosine and Jaccard (e.g., doc1_original.txt - doc6_modified.txt at 0.2665), it could be particularly helpful in scenarios where synonyms or semantically related terms are used. It can identify thematic connections, such as between doc10_paraphrased.txt and doc1_original.txt (0.2507), by linking terms like 'student performance' and 'learning' through their underlying semantic networks, even if the phrasing differs considerably.

Any false positives? Based on the top 5 results for each metric, there were no clear 'false positives' where highly dissimilar documents received high similarity scores. The top pairs identified by Cosine and Jaccard were genuinely lexically similar (original vs. modified versions). WordNet's top scores were relatively modest (around 0.25-0.26), meaning it wasn't incorrectly flagging unrelated documents as highly similar. However, the interpretation of these lower WordNet scores as 'strong matches' depends on the specific application's threshold for semantic similarity.
  
6.conclusion

This lab demonstrates that no single similarity measure is sufficient for robust plagiarism detection.
 Lexical methods like Cosine and Jaccard are effective for direct copying, while semantic methods like WordNet are essential
      for paraphrase detection.
      Combining multiple similarity metrics provides a more reliable and comprehensive analysis of document similarity

1. What is text similarity in NLP?

Text similarity in Natural Language Processing (NLP) measures how similar two pieces of text are in terms of words, structure, or meaning. It is used to identify copied content, paraphrased text, or texts that express similar ideas.

2. Difference between lexical and semantic similarity?

Lexical similarity compares texts based on exact word matches and word frequency.
Example: TF-IDF, Jaccard similarity.

Semantic similarity compares texts based on meaning, even if different words are used.
Example: WordNet similarity.

Lexical methods fail for paraphrased text, while semantic methods can detect meaning similarity.

3. Why is cosine similarity widely used?

Cosine similarity is widely used because it:

Works well with TF-IDF vectors

Is independent of document length

Handles high-dimensional text data efficiently

Provides normalized similarity scores between 0 and 1

This makes it ideal for document comparison.

4. When does Jaccard fail to capture meaning?

Jaccard similarity fails when:

Two texts use different words with the same meaning

Texts are paraphrased

Synonyms are used instead of exact words

Since it only checks word overlap, it cannot understand semantics.

5. How does WordNet improve similarity?

WordNet improves similarity by:

Identifying synonyms and semantic relationships

Comparing word meanings instead of exact words

Detecting paraphrased sentences

It helps measure similarity even when vocabulary differs.

6. How does preprocessing affect similarity scores?

Preprocessing improves similarity accuracy by:

Removing noise like punctuation and stopwords

Normalizing words using lowercasing and lemmatization

Ensuring fair comparison between documents

Without preprocessing, similarity scores may be inaccurate or misleading.

7. Give two real-life applications of text similarity.

Plagiarism detection in academic assignments

Search engines, where similar documents are retrieved for user queries